## Direct solve

In [ ]:
using OptimalControl

# Parameters
Cd = 310.
Tmax = 3.5
β = 500.
b = 2.
N = 100
t0 = 0.
r0 = 1.
v0 = 0.
vmax = 0.1
m0 = 1.
mf = 0.6

@time t
@variable tf
@state x[1:3]
@control u

ocp = Model()

@boundary_constraint(ocp, begin
    x(0) == [ r0, v0, m0 ]
    x(tf)[3] == mf
end)

@state_constraint(ocp, begin
    r0 ≤ r
    0 ≤ v ≤ vmax
    m0 ≤ m ≤ mf
end)

@objective(ocp, Max, r(tf))

function F0(x)
    r, v, m = x
    D = Cd * v^2 * exp(-β*(r-1.))
    F = [ v, -D/m-1.0/r^2, 0. ]
    return F
end

function F1(x)
    r, v, m = x
    F = [ 0., Tmax/m, -b*Tmax ]
    return F
end

@dynamics(ocp, ∂x == F0(x) + u*F1(x))

sol = solve(ocp)
plot(sol)
p0 = sol.adjoint[1] # should also retrieve approximation of switching and final times...

## Indirect solve

In [ ]:
# Bang controls
u0(x, p) = 0.
u1(x, p) = 1.

# Computation of singular control of order 1
H0(x, p) = p' * F0(x)
H1(x, p) = p' * F1(x)
H01 = {H0, H1}
H001 = {H0, H01}
H101 = {H1, H01}
us(x, p) = -H001(x, p) / H101(x, p)

# Computation of boundary control
μ0(x, p) = 0.
g(x) = vmax-x[2] # vmax - v ≥ 0
ub(x) = -Ad(F0, g)(x) / Ad(F1, g)(x)
μb(x, p) = H01(x, p) / Ad(F1, g)(x)

reset!(ocp, :state_constraint)
@state_constraint(ocp, g(x) ≥ 0.)

f0 = Flow(ocp, u0, μ0)
f1 = Flow(ocp, u1, μ0)
fs = Flow(ocp, us, μ0)
fb = Flow(ocp, ub, μb)

# Shooting function
function shoot(x0, p0, t1, t2, t3, tf) # B+ S C B0 structure

    x1, p1 = f1(t0, x0, p0, t1)
    x2, p2 = fs(t1, x1, p1, t2)
    x3, p3 = fb(t2, x2, p2, t3)
    xf, pf = f0(t3, x3, p3, tf)
    s = zeros(eltype(p0), 10)
    s[1:6] = boundary_constraints(ocp, x0, p0, xf, pf)
    s[7] = H1(x1, p1)
    s[8] = H01(x1, p1)
    s[9] = g(x2)
    s[10] = H0(xf, pf) # free tf

    return s

end